<img src="https://github.com/nicholasmetherall/digital-earth-pacific-macblue-activities/blob/main/attachments/images/DE_Pacific_banner.JPG?raw=true" width="900"/>

Figure 1.1.a. Jupyter environment + Python notebooks

# Digital Earth Pacific Notebook 1 prepare postcard and load data to csv

The objective of this notebook is to prepare a geomad postcard for your AOI (masking, scaling and loading additional band ratios and spectral indices) and sampling all the datasets into a csv based on your training data geodataframe.

## Step 1.1: Configure the environment

In [1]:
from datetime import datetime
import geopandas as gpd
import numpy as np
import pandas as pd
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [2]:
# Predefined variable for title and version

# Enter your initials
initials = "nm"

# Date
date = datetime.now()

# Make a clean version string
version = f"{initials}-{date.strftime('%d%m%Y')}"
print(version)

nm-17072025


### Postcard csv

The objective of this notebook was to train the machine learning model that will allow us to classify an area with land cover classes defined through the training data.

Step 1.2. Input the training data to sample geomad data from the postcard

In [3]:
joined_df = gpd.read_file("training-data/nm-17072025-joined_tdata.csv")
joined_df = joined_df.astype("float32")
joined_df

,cc_id,nir,red,blue,green,emad,smad,bcmad,nir08,nir09,...,ln_bg,contrast,homogeneity,energy,ASM,correlation,mean,entropy,y,x
0,3.0,0.1412,0.1573,0.1374,0.1776,0.263184,0.000005,0.000023,0.1332,0.1158,...,-0.256637,8.583333,0.440308,0.128800,0.016590,0.905414,20.875000,6.100209,-2054025.0,3098525.0
1,3.0,0.1412,0.1573,0.1374,0.1776,0.263184,0.000005,0.000023,0.1332,0.1158,...,-0.256637,3.847222,0.511532,0.138889,0.019290,0.953763,20.979166,5.890217,-2054025.0,3098515.0
2,3.0,0.1067,0.1438,0.1382,0.1734,0.227629,0.000006,0.000026,0.1027,0.1134,...,-0.226899,1.791667,0.616912,0.170952,0.029225,0.986131,17.576389,5.529961,-2054045.0,3098515.0
3,3.0,0.1154,0.1662,0.1609,0.1999,0.260504,0.000005,0.000026,0.1122,0.1117,...,-0.217034,0.986111,0.690278,0.227371,0.051698,0.992316,15.673611,4.946617,-2054055.0,3098505.0
4,3.0,0.1160,0.1688,0.1641,0.2024,0.269488,0.000005,0.000028,0.1153,0.1242,...,-0.209770,1.111111,0.677778,0.228006,0.051987,0.991332,15.555555,4.948634,-2054055.0,3098495.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
781,2.0,0.0658,0.1405,0.1520,0.1848,0.134837,0.000003,0.000026,0.0566,0.0564,...,-0.195394,1.027778,0.802778,0.660343,0.436053,0.603512,2.597222,2.238186,-1057695.0,-302245.0
782,2.0,0.0449,0.0928,0.1084,0.1283,0.105597,0.000002,0.000024,0.0417,0.0549,...,-0.168543,1.083333,0.775000,0.572654,0.327932,0.780471,3.069444,2.626067,-1057665.0,-302255.0
783,2.0,0.0431,0.0950,0.1181,0.1391,0.111688,0.000003,0.000020,0.0437,0.0531,...,-0.163661,0.791667,0.770833,0.589174,0.347126,0.561515,2.493056,2.482144,-1057635.0,-302235.0
784,2.0,0.0487,0.1173,0.1420,0.1701,0.115321,0.000002,0.000021,0.0433,0.0534,...,-0.180559,0.666667,0.800000,0.628846,0.395448,0.561866,2.444444,2.200057,-1057645.0,-302235.0


In [4]:
joined_df=joined_df.drop(columns=["y", "x"])

In [5]:
print(len(joined_df.columns))
joined_df.columns

35


Index(['cc_id', 'nir', 'red', 'blue', 'green', 'emad', 'smad', 'bcmad',
       'nir08', 'nir09', 'swir16', 'swir22', 'coastal', 'rededge1', 'rededge2',
       'rededge3', 'mndwi', 'ndti', 'cai', 'ndvi', 'evi', 'savi', 'ndwi',
       'b_g', 'b_r', 'mci', 'ndci', 'ln_bg', 'contrast', 'homogeneity',
       'energy', 'ASM', 'correlation', 'mean', 'entropy'],
      dtype='object')

In [6]:
# The classes are the first column
classes = np.array(joined_df)[:, 0]

# The observation data is everything after the second column
observations = np.array(joined_df)[:, 1:]

# Create a model...
classifier = RandomForestClassifier()

# ...and fit it to the data
model = classifier.fit(observations, classes)

In [7]:
# Dynamically create the filename with f-string
file_path = f"models/{version}-test.model"

# Save the model
joblib.dump(model, file_path)

['models/nm-17072025-test.model']